# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn import linear_model
from sklearn.metrics import mean_squared_error

## Загрузка и подготовка данных

Подготовка данных: 
- Загрузить данные
- Проверить наличие дубликатов
- Удалить NaN
- Посмотреть на тип данных признаков - если есть качественные признаки, превратить их в количественные с помощью OHE
- Отбросить id скважины - этот признак нам не понадобится в ходе обучения модели

In [ ]:
df = pd.read_csv('/datasets/geo_data_0.csv')


In [ ]:
df_1 = pd.read_csv('/datasets/geo_data_1.csv')


In [ ]:
df_2 = pd.read_csv('/datasets/geo_data_2.csv')


In [ ]:
def data_overview(data):
    display(data.head())
    display(data.info())
    print("Пропуски:", data.isna().sum())
    print("Явные дубликаты:", data.duplicated().sum())
    print('')
    colnames = df.select_dtypes('number').columns
    print("Дубликаты по ID:", data['id'].duplicated().sum())
    print(data[data['id'].duplicated()])
    for name in colnames:
            data[name].plot.hist()
            plt.title(name)
            plt.show()
            
    print("Таблица корреляций Пирсона для всех численных переменных")
    print(data.corr())
    print('')
    print("Диаграммы рассеяния для всех численных переменных")
    
    for i in range(0,4,1):
        for j in reversed(range(0,4,1)):
            if i != j:
                data.plot.scatter(x = colnames[i], y = colnames[j])
                plt.xlabel(colnames[i])
                plt.ylabel(colnames[j])
                plt.show()

In [ ]:
datas = [df, df_1, df_2]


In [ ]:
count = 0
for data in datas:
    print('Регион', count)
    data_overview(data)
    count+=1
    print('--------------------------------------------')
    print('')

In [ ]:
#удаляем дубликаты по ИД скважины
for data in datas:
    data = data.drop_duplicates(subset = 'id', inplace = True)

In [ ]:
df['id'].duplicated().sum()

In [ ]:
df_0 = df.drop('id', axis = 1)
df_1 = df_1.drop('id', axis = 1)
df_2 = df_2.drop('id', axis = 1)

**Выводы по подготовке данных:**

- Все нужные признаки количественного типа
- Судя по значениям признаков, они уже приведены к одному масштабу
- Нет пропусков
- Были обнаружены дубликаты по идентификатору скважины. Так как их оказалось мало (всего 4, 4 и 10 для трех датасетов) - мы их просто удалили

После построения матрицы корреляций между признаками, мы обнаружили, что признаки не коррелируют между собой, а наибольший коэффициент корреляции с целевым признаком показывает признак f2, причем во втором датасете (Регион 1) значение Pearson's r равняется 0.99, в то время как в других двух других 0.44 и 0.48. Чтобы понять откуда взялать такая аномалия, мы построили диаграммы рассеяния для всех признаков всех трех датасетов. 


## Обучение и проверка модели

### Регион 0

Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.

In [ ]:
features = df_0.drop(['product'], axis=1)
target = df_0['product']

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(features,target, test_size=0.25, random_state=12345)

Обучите модель и сделайте предсказания на валидационной выборке.

In [ ]:
model = linear_model.LinearRegression()

In [ ]:
model.fit(features_train, target_train)

In [ ]:
model.predict(features_valid)

Сохраните предсказания и правильные ответы на валидационной выборке. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.

In [ ]:
predict_d0 = model.predict(features_valid)
true_d0 = target_valid.reset_index(drop = True)

In [ ]:
#r2_0 = model.score(features_valid, target_valid)
#print('R2 для региона 0 =', r2_0)
rmse_0 = mean_squared_error(true_d0, predict_d0)**0.5
print('RMSE для региона 0 =', rmse_0)
mean_pred_0 = predict_d0.mean()
print('Средний запас предсказанного сырья для региона 0 =', mean_pred_0)

### Регион 1

In [ ]:
features = df_1.drop(['product'], axis=1)
target = df_1['product']

features_train, features_valid, target_train, target_valid = train_test_split(features,target, test_size=0.25, random_state=12345)

model = linear_model.LinearRegression()
model.fit(features_train, target_train)

predict_d1 = model.predict(features_valid)
true_d1 = target_valid.reset_index(drop = True)

#r2_1 = model.score(features_valid, target_valid)
#print('R2 для региона 1 =', r2_1)

rmse_1 = mean_squared_error(true_d1, predict_d1)**0.5
print('RMSE для региона 1 =', rmse_1)
mean_pred_1 = predict_d1.mean()
print('Средний запас предсказанного сырья для региона 1 =', mean_pred_1)

### Регион 2

In [ ]:
features = df_2.drop(['product'], axis=1)
target = df_2['product']

features_train, features_valid, target_train, target_valid = train_test_split(features,target, test_size=0.25, random_state=12345)

model = linear_model.LinearRegression()
model.fit(features_train, target_train)

predict_d2 = model.predict(features_valid)
true_d2 = target_valid.reset_index(drop = True)

#r2_2 = model.score(features_valid, target_valid)
#print('R2 для региона 2 =', r2_2)

rmse_2 = mean_squared_error(true_d2, predict_d2)**0.5
print('RMSE для региона 2 =', rmse_2)
mean_pred_2 = predict_d2.mean()
print('Средний запас предсказанного сырья для региона 2 =', mean_pred_2)

**Выводы после обучения моделей для каждого региона:**

- У региона 0 высокие показатели среднего запаса предсказанного сырья (92), но зато и высокая метрика RMSE (37)
- У региона 1 метрика RMSE почему-то аномально низкая (0.89!), средний запас предсказанного сырья при этом ниже, чем у регионов 0 и 2 (68)
- У региона 2 метрика показатели среднего запаса предсказанного сырья выше всего (95), при этом RMSE также выше регионов 1 и 0 (40)

## Подготовка к расчёту прибыли

Все ключевые значения для расчётов сохраните в отдельных переменных.

In [ ]:
explored_n = 500 #
best_n = 200 # выбирают 200 лучших для разработки.
budget = 10000000000 # Бюджет на разработку скважин в регионе
unit_price = 450000 #Доход с каждой единицы продукта составляет 450 тыс. р, объём указан в тыс баррелей.
max_risk = 0.025 #После оценки рисков нужно оставить лишь те регионы, 
#в которых вероятность убытков меньше 2.5%. 

#Среди них выбирают регион с наибольшей средней прибылью.

Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе. 

In [ ]:
#(enough_units * best_n * unit_price) - budget >= 0
# enough_units * best_n * unit_price = budget
# enough_units = budget/(best_n * unit_price)
enough_units = budget/(best_n * unit_price) 
print(enough_units)

In [ ]:
print(mean_pred_0 >= enough_units)
print(mean_pred_1 >= enough_units)
print(mean_pred_2 >= enough_units)

**Выводы**: Средние предсказанные значения на валидационной выборке ниже необходимого для всех трех регионов.

----

Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- Выберите скважины с максимальными значениями предсказаний.
- Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
- Рассчитайте прибыль для полученного объёма сырья.

In [ ]:
def profits(predicted, true):
    #случайно отберем 500 скважин по условию задачи (explored_n)
    predicted = pd.Series(predicted)
    true = pd.Series(true)
    predicted = predicted.sample(n = explored_n, random_state = 12345, replace = False)
    true = true[predicted.index]
    
    #сбросим индекс
    predicted.reset_index(drop = True, inplace = True)
    true.reset_index(drop = True, inplace = True)
    
    #отберем 200 лучших скважин (best_n) по предсказанию модели
    best = predicted.sort_values(ascending = False)[:best_n]
    #возьмем истинные значения объема этих скважин
    best_true = true[best.index]
    
    profits = (sum(best_true)  * unit_price) - budget
    return profits, best_true

In [ ]:
profits_d0, best_true_d0 = profits(predict_d0, true_d0)
print("Прогноз прибыли для региона 0 =",profits_d0/1000000000, 'в миллиардах рублей')
profits_d1, best_true_d1 = profits(predict_d1, true_d1)
print("Прогноз прибыли для региона 1 =",profits_d1/1000000000, 'в миллиардах рублей')
profits_d2, best_true_d2 = profits(predict_d2, true_d2)
print("Прогноз прибыли для региона 2 =",profits_d2/1000000000, 'в миллиардах рублей')

## Расчёт прибыли и рисков 

Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [ ]:
samples = 1000
state = np.random.RandomState(12345)

In [ ]:
def bootstrap_profits(predicted, true, sample_n):
    profit_dist = []
    current_profit = 0
    for i in range(sample_n):
        predicted = pd.Series(predicted)
        true = pd.Series(true)
        sampled_predicted = predicted.sample(n = 500, replace = True, random_state = state, ignore_index = False)
        sampled_true = true[sampled_predicted.index]
        sampled_predicted.reset_index(drop = True,inplace = True)
        sampled_true.reset_index(drop = True,inplace = True)
        current_profit = profits(sampled_predicted, sampled_true)[0]
        profit_dist.append(current_profit)
    
    return pd.Series(profit_dist)

------
Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

------

In [ ]:
profits_d0_bstr = bootstrap_profits(predict_d0, true_d0, samples)

In [ ]:
plt.hist(profits_d0_bstr)
xcoords = [profits_d0_bstr.quantile(q = 0.025), profits_d0_bstr.quantile(q = 0.975), profits_d0_bstr.mean()]
for xc in xcoords:
    plt.axvline(x=xc)
print("Средняя выручка региона 0 =", profits_d0_bstr.mean()/1000000, 'миллионов рублей')
print("Доверительный интервал 95% для региона 0: от", \
      profits_d0_bstr.quantile(q = 0.025).round(2)/1000000, 'миллионов рублей',\
      "до",profits_d0_bstr.quantile(q = 0.975).round(2)/1000000, 'миллионов рублей')
loss_risk_d0 = (len(profits_d0_bstr[profits_d0_bstr < 0]) / len(profits_d0_bstr))
print("Вероятность убытков для региона 0 =", loss_risk_d0*100, "%")

In [ ]:
profits_d1_bstr = bootstrap_profits(predict_d1, true_d1, samples)

In [ ]:
plt.hist(profits_d1_bstr)
xcoords = [profits_d1_bstr.quantile(q = 0.025), profits_d1_bstr.quantile(q = 0.975), profits_d1_bstr.mean()]
for xc in xcoords:
    plt.axvline(x=xc)
print("Средняя выручка региона 1 =", profits_d1_bstr.mean()/1000000, 'миллионов рублей')
print("Доверительный интервал 95% для региона 1: от",\
      profits_d1_bstr.quantile(q = 0.025).round(2)/1000000, 'миллионов рублей',\
      "до",profits_d1_bstr.quantile(q = 0.975).round(2)/1000000, 'миллионов рублей')
loss_risk_d1 = (len(profits_d1_bstr[profits_d1_bstr < 0]) / len(profits_d1_bstr))
print("Вероятность убытков для региона 1 =", loss_risk_d1*100, "%")

In [ ]:
profits_d2_bstr = bootstrap_profits(predict_d2, true_d2, samples)

In [ ]:
plt.hist(profits_d2_bstr)
xcoords = [profits_d2_bstr.quantile(q = 0.025), profits_d2_bstr.quantile(q = 0.975), profits_d2_bstr.mean()]
for xc in xcoords:
    plt.axvline(x=xc)
print("Средняя выручка региона 2 =", profits_d2_bstr.mean()/1000000, 'миллионов рублей')
print("Доверительный интервал 95% для региона 2: от",\
      profits_d2_bstr.quantile(q = 0.025).round(2)/1000000, 'миллионов рублей',\
      "до",profits_d2_bstr.quantile(q = 0.975).round(2)/1000000, 'миллионов рублей')
loss_risk_d2 = (len(profits_d2_bstr[profits_d2_bstr < 0]) / len(profits_d2_bstr))
print("Вероятность убытков для региона 2 =", loss_risk_d2*100, "%")

**Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.**

Мы предлагаем выбрать для разработки регион 1, так как риск убытков у него значительно ниже, чем у двух других регионов (0.89 %, против 8% у региона 0 и 8.7% у региона 2). При этом средняя и максимальная пронозируемая прибыль у него выше, чем у двух других регионов.